# 7 Solving systems of linear equations

In [ ]:
from math import pi, sqrt, cos, sin, atan2
from random import randint, uniform

import matplotlib.pyplot as plt
import numpy as np
import pygame

import vectors
from vectors import distance

## 7.1 Designing an arcade game

### 7.1.1 Modeling the game

In [ ]:
class PolygonModel():
    def __init__(self,points):
        self.points = points
        self.rotation_angle = 0
        self.x = 0
        self.y = 0

class Ship(PolygonModel):
    def __init__(self):
        super().__init__([(0.5,0), (-0.25,0.25), (-0.25,-0.25)])

class Asteroid(PolygonModel):
    def __init__(self):
        sides = randint(5,9)
        vs = [vectors.to_cartesian((uniform(0.5,1.0), 2 * pi * i / sides))
                for i in range(0,sides)]
        super().__init__(vs)

### 7.1.2 Rendering the game

In [ ]:
ship = Ship()

asteroid_count = 10
asteroids = [Asteroid() for _ in range(0,asteroid_count)]

for ast in asteroids:
    ast.x = randint(-9,9)
    ast.y = randint(-9,9)

GREEN = (  0, 255,   0)
def draw_poly(screen, polygon_model, color=GREEN):
    pixel_points = [to_pixels(x,y) for x,y in polygon_model.transformed()]
    pygame.draw.aalines(screen, color, True, pixel_points, 10)

### 7.1.3 Shooting the laser

In [ ]:
def Ship__laser_segment(self):
    dist = 20. * sqrt(2)
    x,y = self.transformed()[0]
    return (x,y), (x + dist * cos(self.rotation_angle), y + dist*sin(self.rotation_angle))

Ship.laser_segment = Ship__laser_segment

In [ ]:
def check_fire_laser():
    laser = ship.laser_segment()
    keys = pygame.key.get_pressed()
    if keys[pygame.K_SPACE]:
        draw_segment(screen, *laser)
        for asteroid in asteroids:
            if asteroid.does_intersect(laser):
                asteroids.remove(asteroid)

### 7.1.4 Exercise

**Exercise 7.1** Implement `PolygonModel.transformed()` method

In [ ]:
def PolygonModel__transformed(self):
    rotated = [vectors.rotate2d(self.rotation_angle, v) for v in self.points]
    return [vectors.add((self.x,self.y),v) for v in rotated]

PolygonModel.transformed = PolygonModel__transformed

**Exercise 7.2** Write function `to_pixels(x,y)`

In [ ]:
width, height = 400, 400
def to_pixels(x,y):
    return (width/2 + width * x / 20, height/2 - height * y / 20)

### pygame code

Helper functions

In [ ]:
WHITE = (255, 255, 255)
RED =   (255,   0,   0)

def init_pygame():
    global screen, done, clock

    pygame.init()
    screen = pygame.display.set_mode([width,height])
    pygame.display.set_caption("Asteroids!")
    done = False
    clock = pygame.time.Clock()

def start_loop():
    global done, milliseconds

    for event in pygame.event.get(): # User did something
        if event.type == pygame.QUIT: # If user clicked close
            done=True # Flag that we are done so we exit this loop

    clock.tick()
    milliseconds = clock.get_time()

    screen.fill(WHITE)

def draw_segment(screen, v1,v2,color=RED):
    pygame.draw.aaline(screen, color, to_pixels(*v1), to_pixels(*v2), 10)

def draw_objects():
    draw_poly(screen,ship)
    for asteroid in asteroids:
        draw_poly(screen, asteroid, color=GREEN)

Drawing of the ship and asteroids

In [ ]:
init_pygame()
while not done:
    start_loop()
    draw_objects()
    pygame.display.flip()
pygame.quit()

## 7.2 Finding intersection points of lines

### 7.2.5 Deciding whether the laser hits an asteroid

In [ ]:
def intersection(u1,u2,v1,v2):
    a1, b1, c1 = standard_form(u1,u2)
    a2, b2, c2 = standard_form(v1,v2)
    m = np.array(((a1,b1),(a2,b2)))
    c = np.array((c1,c2))
    return np.linalg.solve(m,c)

In [ ]:
# Will fail if lines are parallel!
def do_segments_intersect(s1,s2):
    u1,u2 = s1
    v1,v2 = s2
    l1, l2 = distance(*s1), distance(*s2)
    x,y = intersection(u1,u2,v1,v2)
    return (distance(u1, (x,y)) <= l1 and
            distance(u2, (x,y)) <= l1 and
            distance(v1, (x,y)) <= l2 and
            distance(v2, (x,y)) <= l2)

In [ ]:
def PolygonModel__does_intersect(self, other_segment):
    for segment in self.segments():
        if do_segments_intersect(other_segment,segment):
            return True
    return False

PolygonModel.does_intersect = PolygonModel__does_intersect

### 7.2.6 Identifying unsolvable systems

In [ ]:
def do_segments_intersect(s1,s2):
    u1,u2 = s1
    v1,v2 = s2
    d1, d2 = distance(*s1), distance(*s2)
    try:
        x,y = intersection(u1,u2,v1,v2)
        return (distance(u1, (x,y)) <= d1 and
                distance(u2, (x,y)) <= d1 and
                distance(v1, (x,y)) <= d2 and
                distance(v2, (x,y)) <= d2)
    except np.linalg.linalg.LinAlgError:
        return False

### 7.2.7 Exercise

**Exercise 7.11** Write a Python function `standard_form` that takes two vectors
`v1` and `v2` and finds the line `ax + by = c` passing through both of them.

In [ ]:
def standard_form(v1, v2):
    x1, y1 = v1
    x2, y2 = v2
    a = y2 - y1
    b = x1 - x2
    c = x1 * y2 - y1 * x2
    return a,b,c

**Exercise 7.12** For each of the four distance checks in
`do_segments_intersect`, find a pair of line segments that fail one of the checks
but pass the other three checks.

In [ ]:
def segment_checks(s1,s2):
    u1,u2 = s1
    v1,v2 = s2
    l1, l2 = distance(*s1), distance(*s2)
    x,y = intersection(u1,u2,v1,v2)
    return [
        distance(u1, (x,y)) <= l1,
        distance(u2, (x,y)) <= l1,
        distance(v1, (x,y)) <= l2,
        distance(v2, (x,y)) <= l2
    ]

In [ ]:
segment_checks(((-3,0),(-1,0)),((0,-1),(0,1)))

In [ ]:
segment_checks(((1,0),(3,0)),((0,-1),(0,1)))

In [ ]:
segment_checks(((-1,0),(1,0)),((0,-3),(0,-1)))

In [ ]:
segment_checks(((-1,0),(1,0)),((0,1),(0,3)))

**Exercise 7.14** Write a method to decide whether two `PolygonModel` objects collide.

In [ ]:
def PolygonModel__segments(self):
    point_count = len(self.points)
    points = self.transformed()
    return [(points[i], points[(i+1)%point_count])
            for i in range(0,point_count)]

def PolygonModel__does_collide(self, other_poly):
    for other_segment in other_poly.segments():
        if self.does_intersect(other_segment):
            return True
    return False

PolygonModel.segments = PolygonModel__segments
PolygonModel.does_collide = PolygonModel__does_collide

In [ ]:
square1 = PolygonModel([(0,0), (3,0), (3,3), (0,3)])
square2 = PolygonModel([(1,1), (4,1), (4,4), (1,4)])
square3 = PolygonModel([(-3,-3),(-2,-3),(-2,-2),(-3,-2)])
square1.does_collide(square2), square1.does_collide(square3)

### pygame code - final version

This should be equivalent to `asteroids.py`, except without the screenshot feature.

In [ ]:
def PolygonModel__move(self, milliseconds):
    self.rotation_angle += self.angular_velocity * milliseconds / 1000.0

PolygonModel.move = PolygonModel__move

In [ ]:
def init_objects():
    global ship, asteroids

    ship = Ship()

    asteroid_count = 10
    asteroids = [Asteroid() for _ in range(0,asteroid_count)]

    for ast in asteroids:
        ast.x = randint(-9,9)
        ast.y = randint(-9,9)
        ast.rotation_angle = 0
        ast.angular_velocity = uniform(-pi/2,pi/2)

def move_objects():
    keys = pygame.key.get_pressed()

    for ast in asteroids:
        ast.move(milliseconds)

    if keys[pygame.K_LEFT]:
        ship.rotation_angle += milliseconds * (2*pi / 1000)
    if keys[pygame.K_RIGHT]:
        ship.rotation_angle -= milliseconds * (2*pi / 1000)

In [ ]:
def run_game():
    init_objects()
    init_pygame()

    while not done:
        start_loop()
        move_objects()
        check_fire_laser()
        draw_objects()
        pygame.display.flip()
    pygame.quit()

In [ ]:
run_game()

### Alternative intersection detection

The book converts each line segment into the standard form (*`ax + by = c`*)
before detection whether two line segments intersect.
Here is an alternative idea, which uses parametric form of line segments
for intersection detection. Given two line segments *`(u1, u2)`* and *`(v1, v2)`*,
we can write *`u1 + (u2 - u1) * s = v1 + (v2 - v1) * t`*. Solving for *`s`* and *`t`*
yields *`(u2 - u1) * s + (v1 - v2) * t = v1 - u1`*.

In [ ]:
def do_segments_intersect_alt(s1,s2):
    try:
        u1,u2 = s1
        v1,v2 = s2
        m = np.array(((u2[0]-u1[0], v1[0]-v2[0]), (u2[1]-u1[1], v1[1]-v2[1])))
        c = np.array((v1[0]-u1[0], v1[1]-u1[1]))
        s, t = np.linalg.solve(m, c)
        # With the parametric form, it is much easier to determine whether the
        # intersection is within th line segments.
        result = s>=0 and s<=1 and t>=0 and t<=1
    except np.linalg.linalg.LinAlgError:
        result = False

    # In theory, the assert can fail in corner cases due to floating rounding errors.
    # In practice, the probability of failure is negligible.
    assert result == do_segments_intersect(s1,s2)
    return result

In [ ]:
def PolygonModel__does_intersect(self, other_segment):
    for segment in self.segments():
        if do_segments_intersect_alt(other_segment,segment):
            return True
    return False

PolygonModel.does_intersect = PolygonModel__does_intersect

In [ ]:
run_game()

## 7.3 Generalizing linear equations to higher dimensions

#### Figure 7.25 Three planes plotted in Matplotlib

In [ ]:
ax = plt.subplot(1, 1, 1, projection='3d')
x = y = np.array([-6, 6])
x, y = np.meshgrid(x, y)
ax.plot_surface(x, y, x + y + 1, color='b', alpha=0.3)
ax.plot_surface(x, y, 2 * y - 3, color='y', alpha=0.3)
ax.plot_surface(x, y, 2 - x,     color='m', alpha=0.3)
plt.show()

#### Exercise 7.22 Three non-parallel planes that don't share an intersection point

In [ ]:
ax = plt.subplot(1, 1, 1, projection='3d')
x = y = np.array([-6, 6])
x, y = np.meshgrid(x, y)
ax.plot_surface(x, y, -y,        color='b', alpha=0.3)
ax.plot_surface(x, y,  y,        color='y', alpha=0.3)
ax.plot_surface(x, y, 0 * y + 3, color='m', alpha=0.3)
ax.plot([-6, 6], [0, 0], [0, 0], color='black', linewidth=2)
ax.plot([-6, 6], [-3, -3], [3, 3], color='black', linewidth=2)
ax.plot([-6, 6], [3, 3], [3, 3], color='black', linewidth=2)
plt.show()

#### Exercise 7.24 Three planes intersecting at a single point

In [ ]:
ax = plt.subplot(1, 1, 1, projection='3d')
ax.view_init(elev=20, azim=-60)
x = y = np.array([-6, 6])
x, y = np.meshgrid(x, y)
ax.plot_surface(x, y, -y, color='b', alpha=0.3)
ax.plot_surface(x, y,  y, color='y', alpha=0.3)
ax.plot_surface(x, y, -x, color='m', alpha=0.3)
ax.plot([-6, 6], [0, 0], [0, 0], color='black', linewidth=2)
ax.plot([-6, 6], [-6, 6], [6, -6], color='black', linewidth=2)
ax.plot([-6, 6], [6, -6], [6, -6], color='black', linewidth=2)
ax.scatter([0], [0], [0], s=70, color='black')
plt.show()